In [86]:
import csv
import numpy as np
import os
import argparse
import pandas as pd
import glob as gb
import pdb


np.random.seed(3)

# python convert_videossl_listfiles_to_mmaction_format.py ucf101 --src_listfile_dir /home/jinchoi/src/video-data-aug/data/ucf101 --videossl_listfile_dir /home/jinchoi/src/rehab/dataset/action/VideoSSL/datalist/ucf --tgt_listfile_dir /home/jinchoi/src/video-data-aug/data/ucf101/videossl_splits

# python convert_videossl_listfiles_to_mmaction_format.py hmdb51 --src_listfile_dir /home/jinchoi/src/video-data-aug/data/hmdb51 --videossl_listfile_dir /home/jinchoi/src/rehab/dataset/action/VideoSSL/datalist/hmdb --tgt_listfile_dir /home/jinchoi/src/video-data-aug/data/hmdb51/videossl_splits

# python convert_videossl_listfiles_to_mmaction_format.py kinetics100 --src_listfile_dir /home/jinchoi/src/video-data-aug/data/kinetics400 --videossl_listfile_dir /home/jinchoi/src/rehab/dataset/action/VideoSSL/datalist_from_longlong_jing/kinetics --tgt_listfile_dir /home/jinchoi/src/video-data-aug/data/kinetics400/videossl_splits --ext jpg

# python convert_videossl_listfiles_to_mmaction_format.py kinetics100 --src_listfile_dir /home/jinchoi/src/video-data-aug/data/kinetics400 --videossl_listfile_dir /work/vllab/dataset/video_ssl/datalist_from_longlong_jing/kinetics --tgt_listfile_dir /home/jinchoi/src/video-data-aug/data/kinetics400/videossl_splits --ext jpg

# def parse_args():
#     parser = argparse.ArgumentParser(description='Build file list')    
#     parser.add_argument(
#     'dataset',
#     type=str,
#     choices=[
#         'ucf101', 'kinetics100', 'hmdb51'
#     ],
#     help='dataset to be built file list')

#     parser.add_argument(
#         '--src_listfile_dir', type=str, help='source listfile directory')

#     parser.add_argument(
#         '--tgt_listfile_dir', type=str, help='target listfile directory')

#     parser.add_argument(
#         '--videossl_listfile_dir', type=str, help='target listfile directory')

#     parser.add_argument(
#         '--ext',
#         type=str,
#         default='mp4',
#         choices=['jpg', 'mp4'],
#         help='file extensions')

#     args = parser.parse_args()

#     return args


# args = parse_args()

class args:
    dataset = 'kinetics400'
    src_listfile_dir = 'data/kinetics400'
    tgt_listfile_dir = 'data/kinetics400/videossl_splits'
    ext = 'mp4'
    
    
splits = [1]



if args.dataset == 'ucf101':
    ratios = [5, 10, 20, 50]         
    num_classes = '101'
    src_postfix = ''
    phases = ['train']
elif args.dataset == 'hmdb51':
    # ratios = [5, 10, 20, 40, 50, 60, 100]         
    ratios = [100]         
    num_classes = '51'
    src_postfix = ''
    phases = ['train']
elif args.dataset == 'kinetics400':
#     ratios = [5, 10, 20, 50, 100]    
    ratios = [1, 10] 
    num_classes = '400'
    src_postfix = ''
    phases = ['train', 'val']

if not os.path.exists(args.tgt_listfile_dir):
    os.makedirs(args.tgt_listfile_dir)
    
    
root = 'data/kinetics400/annotations'


for phase in ['train']:
    for ratio in ratios:
        if args.ext == 'mp4':
            src_listfile_path = os.path.join(args.src_listfile_dir, 'kinetics400_{}_list_videos.txt{}'.format(phase, src_postfix))
            tgt_listfile_path = os.path.join(args.tgt_listfile_dir, '{}_{}_{}_percent_labeled_videos.txt'.format(args.dataset, phase, ratio))
        elif args.ext == 'jpg':
            src_listfile_path = os.path.join(args.src_listfile_dir, 'kinetics400_{}_list_rawframes.txt'.format(phase))
            tgt_listfile_path = os.path.join(args.tgt_listfile_dir, '{}_{}_{}_percent_labeled_rawframes.txt'.format(args.dataset, phase, ratio))

        df = pd.read_csv(src_listfile_path, header=None, sep=' ')                
        cur_data = df.values
        cur_data_dict = {}
        corrupted_data_list = [] 
        # contruct a dictionary with class as key and the class/vid, # of frames, label as values
        for row in cur_data:
            video_info_list = row[0].split('/')
            class_name = video_info_list[0]
            if class_name == 'data':
                corrupted_data_list.append(video_info_list)
    #         if len(video_info_list) > 2:
    #             print(row[0])
            if class_name not in cur_data_dict:
                cur_data_dict[class_name] = list()
            if args.ext == 'mp4':
                cur_data_dict[class_name].append((row[0], str(row[1])))
            if args.ext == 'jpg':
                cur_data_dict[class_name].append((video_info_list, str(row[1]), str(row[2])))


        selected_data_array = list()
    
        if args.dataset == 'kinetics400':
            select_num_each_case = 6 * ratio
            for class_name, videos in cur_data_dict.items():
                selected_index = np.random.choice(len(videos), size=select_num_each_case, replace=False)
                selected_videos = [list(videos[idx]) for idx in selected_index]
    #             selected_data_dict['class_name'] = 
                selected_data_array += selected_videos

            random.shuffle(selected_data_array)
            selected_df = pd.DataFrame(np.array(selected_data_array))
            selected_df.to_csv(tgt_listfile_path, header=None, index=False, sep=' ')

#     for corrupted_data in corrupted_data_list:
#         corrupted_video_path = os.path.join(os.path.dirname(root), f'videos_{phase}',
#                                        corrupted_data[-2], corrupted_data[-1]+'.mp4')
#         corrupted_extracted_path = os.path.join(os.path.dirname(root), f'rawframes_{phase}',
#                                        corrupted_data[-2], corrupted_data[-1])
#         if os.path.isfile(corrupted_video_path):
#             os.remove(corrupted_video_path)
#             print(corrupted_video_path, 'Removed')
#         else:
#             print(corrupted_video_path, 'is not there')
            
#         if os.path.isdir(corrupted_extracted_path):
#             os.rmdir(corrupted_extracted_path)
#             print(corrupted_extracted_path, 'Removed')
#         else:
#             print(corrupted_extracted_path, 'is not there')


# for split in splits:
#     print('Processing {} split {}'.format(args.dataset, split))
#     for ratio in ratios:

#             print(corrupted_data)
#     df = pd.read_csv(os.path.join(root, f'kinetics_{phase}.csv'), sep=',')                
#     raw_data = df.values
#     vid_list = []
#     for corrupted_video in corrupted_data_list:
#         vid = corrupted_video[-1][:11]
#         vid_list.append(vid)
#     corrupted_df = df[df["youtube_id"].isin(vid_list)]
#     corrupted_df.to_csv(os.path.join(root, f'kinetics_{phase}_corrupted.csv'), index=False, sep=',', quoting=csv.QUOTE_NONE)
#     print(len(corrupted_df))

In [80]:
selected_data_array = list()
for ratio in ratios:
    if args.dataset == 'kinetics400':
        select_num_each_case = 6 * ratio
        for class_name, videos in cur_data_dict.items():
            selected_index = np.random.choice(len(videos), size=select_num_each_case, replace=False)
            selected_videos = [list(videos[idx]) for idx in selected_index]
#             selected_data_dict['class_name'] = 
            selected_data_array += selected_videos
        
        random.shuffle(selected_data_array)
        selected_df = pd.DataFrame(np.array(selected_data_array))
        selected_df.to_csv(os.path.join(root,'videossl_splits', f'kinetics_{phase}_corrupted.csv'), index=False, sep=',', quoting=csv.QUOTE_NONE)
#             print(selected_videos) 
    break